# [1907. Count Salary Categories]('https://leetcode.com/problems/count-salary-categories/description/?envType=study-plan-v2&envId=top-sql-50')

Table: Accounts

<pre>+-------------+------+
| Column Name | Type |
+-------------+------+
| account_id  | int  |
| income      | int  |
+-------------+------+</pre>
account_id is the primary key (column with unique values) for this table.
Each row contains information about the monthly income for one bank account.
 

Write a solution to calculate the number of bank accounts for each salary category. The salary categories are:

"Low Salary": All the salaries strictly less than 20000.
"Average Salary": All the salaries in the inclusive range (20000, 50000).
"High Salary": All the salaries strictly greater than 50000.

The result table must contain all three categories. If there are no accounts in a category, return 0.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Accounts table:
<pre>+------------+--------+
| account_id | income |
+------------+--------+
| 3          | 108939 |
| 2          | 12747  |
| 8          | 87709  |
| 6          | 91796  |
+------------+--------+</pre>
Output: 
<pre>+----------------+----------------+
| category       | accounts_count |
+----------------+----------------+
| Low Salary     | 1              |
| Average Salary | 0              |
| High Salary    | 3              |
+----------------+----------------+</pre>
Explanation: 
Low Salary: Account 2.
Average Salary: No accounts.
High Salary: Accounts 3, 6, and 8.

In [1]:
import pandas as pd

# Pandas schema

data = [[3, 108939], [2, 12747], [8, 87709], [6, 91796]]
accounts = pd.DataFrame(data, columns=['account_id', 'income']).astype({'account_id':'Int64', 'income':'Int64'})

In [2]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import *

#pyspark schema

spark = SparkSession.builder.getOrCreate()

accounts_df = spark.createDataFrame(accounts)
accounts_df.show(truncate=False)

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
23/11/05 12:37:09 WARN Utils: Your hostname, svchost resolves to a loopback address: 127.0.1.1; using 172.18.28.34 instead (on interface eth0)
23/11/05 12:37:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/05 12:37:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+----------+------+
|account_id|income|
+----------+------+
|3         |108939|
|2         |12747 |
|8         |87709 |
|6         |91796 |
+----------+------+


In [3]:
# in Spark Dataframe
def category(income):
    if income < 20000:
        return 'Low Salary'
    elif income > 50000:
        return 'High Salary'
    else:
        return 'Average Salary'


udf_category = F.udf(category, StringType())

accounts_df.unionAll(
    spark.createDataFrame([
        (-1, 1),  #low
        (-1, 20001),  #avg
        (-1, 50001)])  #high
).withColumn('category', udf_category('income')). \
    groupBy('category').agg(F.count('category').alias('accounts_count')). \
    withColumn('accounts_count', F.col('accounts_count') - 1).show()


+--------------+--------------+
|      category|accounts_count|
+--------------+--------------+
|   High Salary|             3|
|    Low Salary|             1|
|Average Salary|             0|
+--------------+--------------+


In [4]:
# in Spark SQL
accounts_df.createOrReplaceTempView('accounts')
spark.sql('''
select category, count(category)-1 as accounts_count from 
(select *, case when income <20000 then "Low Salary" when income >50000 then "High Salary" else "Average Salary" end as category 
from (
select -1 as account_id,1 as income -- low
union all
select -1 as account_id,20001 as income -- high
union all
select -1 as account_id,50001 as income -- avg
union all
select * from accounts
)) group by category
''').show()

+--------------+--------------+
|      category|accounts_count|
+--------------+--------------+
|    Low Salary|             1|
|Average Salary|             0|
|   High Salary|             3|
+--------------+--------------+


In [5]:
# IN spark SQL using udf

accounts_df.createOrReplaceTempView('accounts')
spark.udf.register('udf_category',udf_category)

spark.sql('''
select category, count(category)-1 as accounts_count from 
(select *, udf_category(income) as category 
from (
select -1 as account_id,1 as income -- low
union all
select -1 as account_id,20001 as income -- high
union all
select -1 as account_id,50001 as income -- avg
union all
select * from accounts
)) group by category
''').show()

+--------------+--------------+
|      category|accounts_count|
+--------------+--------------+
|    Low Salary|             1|
|Average Salary|             0|
|   High Salary|             3|
+--------------+--------------+


In [6]:
spark.stop()